In [1]:
from config import *
import tweepy
import psycopg2
import getpass
import json


p = getpass.getpass(prompt="Password: ")
conn = psycopg2.connect(user = "postgres",
                                  password = f"{p}",
                                  host = "localhost",
                                  port = "5432",
                                  database = "twitter_db")

cursor = conn.cursor()

cursor.execute("select * from twitter_feed")

fetch = cursor.fetchone()
print(fetch)
# rds_connection_string = f"postgres:{p}@localhost:5432/twitter_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

Password: ········
None


In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

my_list = []
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
    
    def on_status(self, status):
        collection = []
        print(status.text)
        #Don't know why it doesnt print out coordinates
        print(status.coordinates)
        self.num_tweets += 1
        if self.num_tweets < 20:
            collection.append(status)
            return True
        else:
            return False
        
    def on_data(self, data):
        print("On Data")
        all_data = json.loads(data)
#         print(json.dumps(all_data, indent=4))
        date = all_data['timestamp_ms']
        place = all_data['user']['location']
        tweet = all_data["text"].replace('"',"")
        username = all_data["user"]["screen_name"]
        metadata = all_data['user']["favourites_count"]
        print((place,username,tweet))
        print(f'INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) VALUES ({date},"{place}","{tweet}",{username},{metadata});')
        self.num_tweets += 1
        if self.num_tweets < 20:
            cursor.execute('INSERT INTO twitter_feed ( user_location, user_id, text, metadata) \
            VALUES (%s,%s,%s,%s);', (place,tweet,username,metadata))

            conn.commit()

            print((username,tweet))

            return True
        else:
            return False
    
        


myStreamListener = MyStreamListener()

myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)


myStream.filter(track=['election2020'], is_async=True)

Authentication OK
On Data
('Chelmsford, East', 'Expectivisms', 'RT @williewjewell: Just a #Election2020 reminder #DemocratsAreCorrupt and to #VoteRedToSaveAmerica!\n\nWe need to support our President @real…')
INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) VALUES (1582178316644,"Chelmsford, East","RT @williewjewell: Just a #Election2020 reminder #DemocratsAreCorrupt and to #VoteRedToSaveAmerica!

We need to support our President @real…",Expectivisms,224983);
('Expectivisms', 'RT @williewjewell: Just a #Election2020 reminder #DemocratsAreCorrupt and to #VoteRedToSaveAmerica!\n\nWe need to support our President @real…')
On Data
('Galena Park, TX', 'TrespassingSatr', 'RT @williewjewell: Just a #Election2020 reminder #DemocratsAreCorrupt and to #VoteRedToSaveAmerica!\n\nWe need to support our President @real…')
INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) VALUES (1582178318094,"Galena Park, TX","RT @williewjewell: Just a #Election2020 remi